In [1]:
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', 100)
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Quad
from datetime import datetime
from math import pi
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
import random
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
t = time()

In [2]:
# Keras imports
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, AveragePooling1D, UpSampling1D
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional, TimeDistributed
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, generic_utils
from sklearn.metrics import f1_score
from keras.models import Model
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras import regularizers
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, Callback, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from weightnorm import data_based_init, AdamWithWeightnorm
from keras.utils import np_utils
import keras.backend as K
from itertools import product
from functools import partial
from keras.layers.noise import GaussianNoise
from keras.layers.advanced_activations import ELU
from keras.constraints import maxnorm
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, merge, add
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from sklearn import model_selection
from sklearn.preprocessing import Imputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn import cross_validation
from sklearn.model_selection import KFold
from matplotlib import pyplot
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [5]:
# Import custom scripts and functions
%aimport model_performance_evaluation
%aimport custom_metrics
%aimport data_processing
%aimport make_keras_generators

In [6]:
# Create list of target csvs
data = ["EURJPY1440.csv"]

# allows us to easily modify how many days ahead and before we look:
lookforward = 3 # how many days we look forward
lookback = 7 # how many days we look back

        
# Choose features to be used in feature generation function
feature_generation_channels = ["O", "H", "L", "C", "V"] 

df_ohlc, df_data_unscaled = data_processing.prepare_wild_data(data, 
                                                                 lookforward, 
                                                                 lookback,
                                                                 feature_generation_channels,
                                                                 lag_data=1,
                                                                 change_open_close=1, 
                                                                 range_high_low=1,
                                                                 rolling_mean_range_HL=1,
                                                                 range_high_close=0,
                                                                 std_dev=1,
                                                                 rolling_mean=1,
                                                                 change_open_close_shift=0,
                                                                 range_high_low_shift=0,
                                                                 range_high_close_shift=0,
                                                                 diff_shift=1,
                                                                 rolling_mean_shift=1,
                                                                 std_dev_shift=0,
                                                                 center=False
                                                                )

Processing:  EURJPY1440.csv Number  1  of  1
We now have: (0, 0) rows of data!
Reading in Date, OHLCV, Supply/Demand
Renaming df_ohlc column headers for clarity
Your df_norm columns: Index(['O', 'H', 'L', 'C', 'V'], dtype='object')
Your df_non_norm columns: Index(['Epoch Date'], dtype='object')
Generating features:  Difference between open and close
Shape is now: (4950, 6)
~~~
Generating features:  Range between high and low
Shape is now: (4950, 7)
~~~
Generating features:  Generic lagged data
Generating features:  Diffs over time for lookforward/lookback range
Generating features:  Rolling mean for yesterday, today, tomorrow range
df_norm is: <class 'pandas.core.frame.DataFrame'> (4950, 19)
df_norm_and_features is: <class 'pandas.core.frame.DataFrame'> (4950, 159)
Merging the scaled data and the unscalable data back together with pd.concat: EURJPY1440.csv (4950, 172)
df_merged is: <class 'pandas.core.frame.DataFrame'> (4950, 172)
Finished: EURJPY1440.csv (4950, 172)
------------------

In [7]:
features_to_remove = [
                        'Epoch Date', 
                      'Supply/Demand', 'Object Name', 'Epoch Date Start', 'L_diff_0', 'O_diff_0',
                      'Epoch Date End', 'Proximal', 'Distal', 'change', 'V_diff_0', 'C_diff_0', 'H_diff_0',
                      'O', 'H', 'L', 'C', 
                      'csv_origin_tag', 'range_high_close_range_0', 'change_open_close', 'range_high_close', 
                      'rectangle_here', 
                      'supply_demand', 'group']

lag_cols = [col for col in list(df_ohlc) if col.startswith('lag')]

data_to_save = ['Epoch Date', 'O', 'H', 'L', 'C', 'V']
save_the_date_and_OHLCV = df_ohlc[data_to_save]
lag_data = df_ohlc[lag_cols]
df_ohlc = data_processing.drop_non_features(df_ohlc, features_to_remove, zero_columns=1)



In [8]:
# Resort the DF and add what we think are the more important features to the front of the columns list
# Also, this removes all the plain 'lag' data, which we think has no meaning
X_cols_resort = ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]

X = df_ohlc[X_cols_resort]

# Drop any NaN rows
X = X.dropna(how='any') 

In [9]:
# Create autoencoded features
X_ae = data_processing.autoencode(X)

88


/Users/production204/Github/the-internet-of-forex/data_processing.py:886: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  autoencoder = Model(input = input_dim, output = decoded4)


Train on 2227 samples, validate on 248 samples
Epoch 1/20
2227/2227 [==============================] - 2s - loss: 0.6929 - val_loss: 0.6925
Epoch 2/20
2227/2227 [==============================] - 0s - loss: 0.6923 - val_loss: 0.6919
Epoch 3/20
2227/2227 [==============================] - 0s - loss: 0.6918 - val_loss: 0.6914
Epoch 4/20
2227/2227 [==============================] - 0s - loss: 0.6913 - val_loss: 0.6908
Epoch 5/20
2227/2227 [==============================] - 0s - loss: 0.6907 - val_loss: 0.6903
Epoch 6/20
2227/2227 [==============================] - 0s - loss: 0.6901 - val_loss: 0.6897
Epoch 7/20
2227/2227 [==============================] - 0s - loss: 0.6896 - val_loss: 0.6891
Epoch 8/20
2227/2227 [==============================] - 0s - loss: 0.6889 - val_loss: 0.6884
Epoch 9/20
2227/2227 [==============================] - 0s - loss: 0.6883 - val_loss: 0.6877
Epoch 10/20
2227/2227 [==============================] - 0s - loss: 0.6876 - val_loss: 0.6870
Epoch 11/20
2227/2227 

/Users/production204/Github/the-internet-of-forex/data_processing.py:891: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input = input_dim, output = encoded4)


In [10]:
# Add together the main features and the autoencoder features
X_ = pd.concat([X.reset_index(drop=True), X_ae.reset_index(drop=True)], axis=1, ignore_index=True)  
X_.columns = list(X) + list(X_ae)

In [11]:
# Create a list to re-sort with again, putting our AE features at the front
X__cols_resort = list(X_ae) + ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]
X_ = X_[X__cols_resort]

In [12]:
# Stack original X_train to 3dim for conv net
X_ = data_processing.X_to_Conv1D_arrays(X_)

X:  (4950, 91, 1)
X:  <class 'numpy.ndarray'>


In [13]:
print(X_.shape)

(4950, 91, 1)


In [14]:
df_ohlc = df_ohlc.dropna(how='any') 

## Predict rectangle_here

In [15]:
rectangle_here_model = load_model('rectangle_here-AE-july6-301pm-Conv1D-39-0.63.h5', custom_objects={'AdamWithWeightnorm': AdamWithWeightnorm, 'fbeta_score': custom_metrics.fbeta_score, 'recall': custom_metrics.recall, 'precision': custom_metrics.precision})

In [16]:
rectangle_preds, rectangle_y_pred_proba = model_performance_evaluation.pred(rectangle_here_model,
                                                        X_,
                                                        batch_size=64, 
                                                        sensitivity=0.5, 
                                                        binary=1, 
                                                        sequential=0)

In [17]:
# For lining up predictions and data
df_rectangle_here_predictions = pd.DataFrame(rectangle_preds.astype(int))
df_OHLC_w_rectangle_here = pd.concat([df_rectangle_here_predictions.reset_index(drop=True), 
                                      df_ohlc], 
                         axis=1)
rect_col_label = ['rectangle_here']
df_OHLC_w_rectangle_here.columns = list(rect_col_label) + list(df_ohlc)

In [18]:
df_OHLC_w_rectangle_here.shape


(4950, 156)

## Make Proximal and Distal predictions

#### Predict Proximal

In [19]:
# Make list of features and label column names
reduced_proximal_match_list = ['Proximal_match_1_C', 
                      'Proximal_match_0_C', 
                      'Proximal_match_0_O', 
                      'Proximal_match_-1_O']

# Select only rows that contain zones
df_zones = df_OHLC_w_rectangle_here.loc[df_OHLC_w_rectangle_here['rectangle_here'] == 1]

In [20]:
# Resort the DF and add what we think are the more important features to the front of the columns list
# Also, this removes all the plain 'lag' data, which we think has no meaning
X_cols_resort = ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]

df_zones = df_zones[X_cols_resort]

# Drop any NaN rows (the imputer is clearly broken)
df_zones = df_zones.dropna(how='any') 

# Drop rectangle_here
df_zones = data_processing.drop_non_features(df_zones, ['rectangle_here'])

In [21]:
# Create autoencoded features
X_ae = data_processing.autoencode(df_zones)

88


/Users/production204/Github/the-internet-of-forex/data_processing.py:886: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  autoencoder = Model(input = input_dim, output = decoded4)


Train on 483 samples, validate on 54 samples
Epoch 1/20
483/483 [==============================] - 0s - loss: 0.6930 - val_loss: 0.6929
Epoch 2/20
483/483 [==============================] - 0s - loss: 0.6929 - val_loss: 0.6929
Epoch 3/20
483/483 [==============================] - 0s - loss: 0.6928 - val_loss: 0.6928
Epoch 4/20
483/483 [==============================] - 0s - loss: 0.6927 - val_loss: 0.6927
Epoch 5/20
483/483 [==============================] - 0s - loss: 0.6927 - val_loss: 0.6926
Epoch 6/20
483/483 [==============================] - 0s - loss: 0.6926 - val_loss: 0.6925
Epoch 7/20
483/483 [==============================] - 0s - loss: 0.6925 - val_loss: 0.6925
Epoch 8/20
483/483 [==============================] - 0s - loss: 0.6924 - val_loss: 0.6924
Epoch 9/20
483/483 [==============================] - 0s - loss: 0.6923 - val_loss: 0.6923
Epoch 10/20
483/483 [==============================] - 0s - loss: 0.6923 - val_loss: 0.6922
Epoch 11/20
483/483 [=======================

/Users/production204/Github/the-internet-of-forex/data_processing.py:891: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input = input_dim, output = encoded4)


In [22]:
# Add together the main features and the autoencoder features
X_ = pd.concat([df_zones.reset_index(drop=True), X_ae.reset_index(drop=True)], axis=1, ignore_index=True)  
X_.columns = list(df_zones) + list(X_ae)

In [23]:
# Create a list to re-sort with again, putting our AE features at the front
X__cols_resort = list(X_ae) + ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]
X_ = X_[X__cols_resort]

In [24]:
# Stack original X_train to 3dim for conv net
X_zones = data_processing.X_to_Conv1D_arrays(X_)

X:  (1075, 91, 1)
X:  <class 'numpy.ndarray'>


###### Proximal model

In [25]:
# Load model
proximal_model = load_model('proximal-AE-july6-131pm-LSTM-83-0.84.h5', custom_objects={'AdamWithWeightnorm': AdamWithWeightnorm, 'fbeta_score': custom_metrics.fbeta_score, 'recall': custom_metrics.recall, 'precision': custom_metrics.precision})

In [26]:
#proximal_model.summary()

In [27]:
proximal_preds, y_pred_proba = model_performance_evaluation.pred(proximal_model,
                                                        X_zones,
                                                        batch_size=64, 
                                                        #sensitivity=0.5, 
                                                        temperature=0.8,
                                                        binary=0, 
                                                        sequential=0)

In [28]:
proximal_col_labels = reduced_proximal_match_list
proximal_preds.columns = list(proximal_col_labels)

#### Predict Distal

In [29]:
# Make list of features and label column names
reduced_distal_match_list = ['Distal_match_-1_H', 
                              'Distal_match_0_H', 
                              'Distal_match_1_H', 
                              'Distal_match_2_H', 
                              'Distal_match_-1_L', 
                              'Distal_match_0_L',
                              'Distal_match_1_L',
                              'Distal_match_2_L']

###### Distal model

In [31]:
# Load model
distal_model = load_model('distal-AE-july6-240pm-LSTM-99-0.83.h5', custom_objects={'AdamWithWeightnorm': AdamWithWeightnorm, 'fbeta_score': custom_metrics.fbeta_score, 'recall': custom_metrics.recall, 'precision': custom_metrics.precision})

In [32]:
#distal_model.summary()

In [33]:
distal_preds, distal_y_pred_proba = model_performance_evaluation.pred(distal_model,
                                                        X_zones,
                                                        batch_size=64, 
                                                        sensitivity=0.5, 
                                                        temperature=0.8,
                                                        binary=0, 
                                                        sequential=0)

In [34]:
# For lining up predictions and data

distal_col_labels = reduced_distal_match_list
distal_preds.columns = list(distal_col_labels)

## Convert predictions to discrete values and visualize

In [35]:
proximal_preds.index = df_zones.index

In [36]:
distal_preds.index = df_zones.index

In [37]:
df_ohlc_and_proximal = save_the_date_and_OHLCV.join(proximal_preds)

In [38]:
df_ohlc_and_zone_preds = df_ohlc_and_proximal.join(distal_preds)

In [39]:
df_ohlc_and_zone_preds_w_lag = df_ohlc_and_zone_preds.join(lag_data)

In [40]:
data_processing.proximal_distal_match_invert(df_ohlc_and_zone_preds_w_lag, df_zones.index)

## Visualize preds and OHLC

In [41]:
import zigzag as zigzag

In [42]:
ohlc_data_col_list = ['O',
 'H',
 'L',
 'C',
 'V',
 'Proximal_discrete_value',
 'Distal_discrete_value']

df_ohlc_and_rectangles_col_list = ['Epoch Date',
 'Supply/Demand',
 'Epoch Date Start',
 'Epoch Date End',]

new_ohlc_data_col_list = ['O',
 'H',
 'L',
 'C',
 'V',
 'Proximal',
 'Distal']

In [43]:
labels_distal = [col for col in list(df_ohlc_and_zone_preds_w_lag) if col.startswith('Distal_m')]
labels_proximal = [col for col in list(df_ohlc_and_zone_preds_w_lag) if col.startswith('Proximal_m')]

In [44]:
df_ohlc_data = data_processing.drop_non_features(df_ohlc_and_zone_preds_w_lag, lag_cols + labels_distal + labels_proximal)

In [45]:
df_ohlc_data_test = df_ohlc_data

In [46]:
df_ohlc_data_test_cols = ['Epoch Date',
 'O',
 'H',
 'L',
 'C',
 'V',
'Proximal', 'Distal']

In [47]:
df_ohlc_data_test.columns = df_ohlc_data_test_cols

In [48]:
df_ohlc_data_test['Epoch Date End'] = np.nan
df_ohlc_data_test['Supply/Demand'] = np.nan

In [49]:
data_processing.supply_or_demand(df_ohlc_data_test, df_zones.index)

In [50]:
data_processing.zone_ender(df_ohlc_data_test, df_zones.index)

In [64]:
# Set Epoch Date Start to the zone landing spot
df_ohlc_data_test.ix[df_zones.index, 'Epoch Date Start'] = df_ohlc_data_test.ix[df_zones.index]['Epoch Date']

In [66]:
def viz_ohlc(df):    
    # Convert Dates to datetime
    df["Epoch Date"] = pd.to_datetime(df["Epoch Date"], unit='s')
    df["Epoch Date Start"] = pd.to_datetime(df["Epoch Date Start"], unit='s')
    df["Epoch Date End"] = pd.to_datetime(df["Epoch Date End"], unit='s')

    w = 12*60*60*1000 # half day in ms

    TOOLS = "pan,xwheel_zoom,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type="datetime", tools=TOOLS, plot_height=500, 
               plot_width=1200, title="🍩", active_scroll='xwheel_zoom')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.5

    p.segment(df['Epoch Date'], df.H, df['Epoch Date'], df.L, color="black")
    p.vbar(df['Epoch Date'], w, df.O, df.C)

    # Add Rectangle data
    df_sp = df.dropna(subset=['Supply/Demand'])

    source_supply = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='SUPPLY', :])
    rect_supply = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="firebrick", line_alpha=0.2, fill_alpha=0.2) 
    p.add_glyph(source_supply, rect_supply)

    source_demand = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='DEMAND', :])
    rect_demand = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="steelblue", line_alpha=0.2, fill_alpha=0.2)
    p.add_glyph(source_demand, rect_demand)


    output_file("candlestick.html", title="candlestick data")
    show(p)  # open a browser

In [67]:
viz_ohlc(df_ohlc_data_test)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bokeh/plotting/helpers.py:371: UserWarning: WheelZoomTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))
